In [1]:
%fs ls FileStore/tables

path,name,size
dbfs:/FileStore/tables/MillionSales.csv,MillionSales.csv,124793263


In [2]:
data = spark.read.load('FileStore/tables/MillionSales.csv', format='com.databricks.spark.csv', header='true')

In [3]:
# print schema of dataframe
data.printSchema()

root
-- Region: string (nullable = true)
-- Country: string (nullable = true)
-- Item Type: string (nullable = true)
-- Sales Channel: string (nullable = true)
-- Order Priority: string (nullable = true)
-- Order Date: string (nullable = true)
-- Order ID: string (nullable = true)
-- Ship Date: string (nullable = true)
-- Units Sold: string (nullable = true)
-- Unit Price: string (nullable = true)
-- Unit Cost: string (nullable = true)
-- Total Revenue: string (nullable = true)
-- Total Cost: string (nullable = true)
-- Total Profit: string (nullable = true)

In [4]:
# count numbers in dataframe
data.count()

Out[ 3 ]: 1000000

In [5]:
# probe dataframe
data.limit(5).show()

+--------------------+----------------+---------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
 Region| Country|Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+----------------+---------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
 Sub-Saharan Africa| South Africa| Fruits| Offline| M| 7/27/2012|443368995| 7/28/2012| 1593| 9.33| 6.92| 14862.69| 11023.56| 3839.13|
Middle East and N...| Morocco| Clothes| Online| M| 9/14/2013|667593514|10/19/2013| 4611| 109.28| 35.84| 503890.08| 165258.24| 338631.84|
Australia and Oce...|Papua New Guinea| Meat| Offline| M| 5/15/2015|940995585| 6/4/2015| 360| 421.89| 364.69| 151880.40| 131288.40| 20592.00|
 Sub-Saharan Africa| Djibouti| Clothes| Offline| H| 5/17/2017|880811536| 7/2/2017| 562| 109.28| 35.84| 61415.36| 20142.08| 41273.28|
 Europe| Slovakia|Beverages| Offline| L|10/26/2016|174590194| 12/4/2016| 3973| 47.45| 31.79| 188518.85| 126301.67| 62217.18|
+--------------------+----------------+---------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+

In [6]:
data.sample(fraction=0.01).count()

Out[ 31 ]: 10117

In [7]:
# this will exclude column from dataframe
data.drop('Sales Channel')

+--------------------+----------------+---------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
 Region| Country|Item Type|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+----------------+---------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
 Sub-Saharan Africa| South Africa| Fruits| M| 7/27/2012|443368995| 7/28/2012| 1593| 9.33| 6.92| 14862.69| 11023.56| 3839.13|
Middle East and N...| Morocco| Clothes| M| 9/14/2013|667593514|10/19/2013| 4611| 109.28| 35.84| 503890.08| 165258.24| 338631.84|
Australia and Oce...|Papua New Guinea| Meat| M| 5/15/2015|940995585| 6/4/2015| 360| 421.89| 364.69| 151880.40| 131288.40| 20592.00|
 Sub-Saharan Africa| Djibouti| Clothes| H| 5/17/2017|880811536| 7/2/2017| 562| 109.28| 35.84| 61415.36| 20142.08| 41273.28|
 Europe| Slovakia|Beverages| L|10/26/2016|174590194| 12/4/2016| 3973| 47.45| 31.79| 188518.85| 126301.67| 62217.18|
+--------------------+----------------+---------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+

In [8]:
# this will select only Countries and create distinct and sorted list of it
data.select('Country').distinct().orderBy('Country').show()

+--------------------+
 Country|
+--------------------+
 Afghanistan|
 Albania|
 Algeria|
 Andorra|
 Angola|
Antigua and Barbuda |
 Armenia|
 Australia|
 Austria|
 Azerbaijan|
 Bahrain|
 Bangladesh|
 Barbados|
 Belarus|
 Belgium|
 Belize|
 Benin|
 Bhutan|
Bosnia and Herzeg...|
 Botswana|
+--------------------+
only showing top 20 rows

In [9]:
# this one select two columns with distinct values and then filter it by given country
data.select('Country','Item Type').distinct().filter(data['Country']=='Poland')
# is in filter
data.select('Country','Item Type').filter(data['Country'].isin('Poland','Germany')).distinct().orderBy('Item Type').show()

+-------+---------------+
Country| Item Type|
+-------+---------------+
 Poland|Office Supplies|
 Poland| Clothes|
 Poland| Beverages|
 Poland| Cosmetics|
 Poland| Snacks|
 Poland| Cereal|
 Poland| Household|
 Poland| Baby Food|
 Poland| Vegetables|
 Poland| Meat|
 Poland| Personal Care|
 Poland| Fruits|
+-------+---------------+

In [10]:
# Sales grouped by Country
grouped = data.groupBy('Country').count()
# count it's an aggregation functions
# to get number of grouped elements twice 'count' call is necessary:
numberGrouped = data\
                  .groupBy('Country')\
                  .count()\
                  .count()
# if structure of query become too long for one line \ sign nad new line can be applied to move parts of query to new line.

In [11]:
# to perform sum aggregation
#data.groupBy('Country').agg({"Total Profit":"sum"}).orderBy('Country').show()
# to perform avg aggregation
data.groupBy('Country').agg({"Total Profit":"avg"}).orderBy('Country').show()

+--------------------+------------------+
 Country| avg(Total Profit)|
+--------------------+------------------+
 Afghanistan| 394033.1363970453|
 Albania| 390933.3249944568|
 Algeria| 387027.2472598768|
 Andorra|384667.91692207556|
 Angola|383484.09701315535|
Antigua and Barbuda |393922.75622722215|
 Armenia| 390654.4262224263|
 Australia|397394.40966922534|
 Austria| 388575.1625432657|
 Azerbaijan| 387611.7719914385|
 Bahrain| 398206.2895791988|
 Bangladesh| 398636.8292591214|
 Barbados| 390276.1127024501|
 Belarus| 379621.0808547328|
 Belgium|390461.24328238587|
 Belize|388222.80973461334|
 Benin|394359.87549493817|
 Bhutan|393809.68077362305|
Bosnia and Herzeg...| 394588.3302426376|
 Botswana| 389864.670580514|
+--------------------+------------------+
only showing top 20 rows